In [1]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
import itertools
import yfinance as yf
import seaborn as sn
from IPython.core.display import display, HTML
from trend_following import (jupyter_interactive_mode, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             sharpe_ratio, create_trend_strategy, get_close_prices)

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
jupyter_interactive_mode()

## Helper functions to help pull the data and run the analysis

## Pull the data from Yahoo Finance

We begin by pulling daily data for the specified instrument using the Yahoo finance API. The data available only goes back to late 2017. 

In [61]:
# Specify date range and pull data for specific instrument
start_date = pd.datetime(2021, 1, 1)
end_date = pd.datetime(2024, 7, 30)
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
# ticker = ['TSLA']
data = get_close_prices(start_date, end_date, ticker)

File data found...reading ['BCH-USD', 'ETH-USD', 'USDT-USD', 'XRP-USD', 'BNB-USD', 'BTC-USD'] data


In [4]:
def calculate_ts_momentum(df, close_px_col, lookback_period, period_name, returns_type='arithmetic'):
    
    if returns_type == 'logarithmic':
        df[f'{close_px_col}_log_returns'] = np.log(df[close_px_col]/df[close_px_col].shift(1))
        df[f'{close_px_col}_{period_name}_momentum'] = df[f'{close_px_col}_log_returns'].rolling(window=lookback_period).sum()
    elif returns_type == 'arithmetic':
        df[f'{close_px_col}_pct_returns'] = df[f'{close_px_col}'].pct_change()
        df[f'{close_px_col}_{period_name}_momentum'] = df[f'{close_px_col}_pct_returns'].rolling(window=lookback_period).sum()
    
    return df

## CHECK THIS, THIS IS NON-COMPOUNDED RETURNS
def calculate_cumulative_returns(df, close_px_col, returns_type='logarithmic'):
    
    if returns_type == 'logarithmic':
        df[f'{close_px_col}_log_returns'] = np.log(df[close_px_col]/df[close_px_col].shift(1))
        df[f'{close_px_col}_log_cum_returns'] = df[f'{close_px_col}_log_returns'].cumsum()
    elif returns_type == 'arithmetic':
        df[f'{close_px_col}_pct_returns'] = df[f'{close_px_col}'].pct_change()
        df[f'{close_px_col}_pct_cum_returns'] = df[f'{close_px_col}_pct_returns'].cumsum()
    
    return df

In [5]:
def get_long_and_short_term_momentum(df, close_px_col, returns_type, weekend_trading=False):
    
    if weekend_trading:
        trading_lookback_periods = [10, 21, 63, 126, 252]
    else:
        trading_lookback_periods = [10, 30, 90, 180, 365]
        
    lookback_period_names = ['10_day', '1_month', '3_month', '6_month', '12_month']
    
    c = 0
    for lookback in trading_lookback_periods:
        df = calculate_ts_momentum(df, close_px_col, lookback_period=lookback, period_name=lookback_period_names[c], returns_type=returns_type)
        c+=1
    
    return df

In [6]:
def generate_momentum_signal(df, ticker, momentum_type, threshold):
    
    df[f'{ticker}_{momentum_type}_signal'] = np.where(df[f'{ticker}_{momentum_type}'] > threshold, 1,
                                             np.where(df[f'{ticker}_{momentum_type}'] < -threshold, -1, np.nan))
    ## IS THIS REQUIRED
    df[f'{ticker}_{momentum_type}_signal'] = df[f'{ticker}_{momentum_type}_signal'].shift(1)
    
    df[f'{ticker}_daily_returns'] = df[f'{ticker}'].pct_change()
    
    df[f'{ticker}_{momentum_type}_strategy_returns'] = df[f'{ticker}_{momentum_type}_signal'] * df[f'{ticker}_daily_returns']
    
    # Calculate cumulative returns
    df[f'{ticker}_{momentum_type}_cum_strategy_returns'] = (1 + df[f'{ticker}_{momentum_type}_strategy_returns']).cumprod() - 1
    df[f'{ticker}_{momentum_type}_cum_buy_and_hold_returns'] = (1 + df[f'{ticker}_daily_returns']).cumprod() - 1
    
    return df

In [21]:
def calculate_drawdown(df, strategy_daily_return_col):
    
    df['strategy_cumulative_return'] = (1 + df[strategy_daily_return_col]).cumprod() - 1
    df[f'strategy_cumulative_return_cum_max'] = df['strategy_cumulative_return'].cummax()
    df[f'strategy_cumulative_return_drawdown'] = df['strategy_cumulative_return'] - df[f'strategy_cumulative_return_cum_max']
    df[f'strategy_cumulative_return_drawdown_pct'] = df[f'strategy_cumulative_return_drawdown'] / df[f'strategy_cumulative_return_cum_max']
    
    # Calculate maximum drawdown
    max_drawdown = df[f'strategy_cumulative_return_drawdown_pct'].min()
    
    # Calculate maximum drawdown duration
    df['End'] = df.index
    df['Start'] = df[f'strategy_cumulative_return_cum_max'].ne(df[f'strategy_cumulative_return_cum_max'].shift(1)).cumsum()
    df[f'strategy_cumulative_return_DDDuration'] = df.groupby('Start')['End'].transform(lambda x: x.max() - x.min())
    max_drawdown_duration = df[f'strategy_cumulative_return_DDDuration'].max()

    # Drop NaN values for better display
#     df = df.dropna(inplace=True)
    return df, max_drawdown, max_drawdown_duration

In [56]:
def calculate_CAGR(df, strategy_daily_return_col, annual_trading_days=252):
    
    # Calculate cumulative return
    df['strategy_cumulative_return'] = (1 + df[strategy_daily_return_col]).cumprod() - 1

    # Calculate the total cumulative return at the end of the period
    total_cumulative_return = df['strategy_cumulative_return'].iloc[-1]

    # Calculate the number of periods (days)
    num_periods = len(df)

    # Convert the number of periods to years (assuming daily data, 252 trading days per year)
    trading_days_per_year = annual_trading_days
    num_years = num_periods / trading_days_per_year

    # Calculate CAGR
    annualized_return = (1 + total_cumulative_return) ** (1 / num_years) - 1

    # Convert the annualized return to a percentage
#     annualized_return_percentage = annualized_return * 100
    
    return annualized_return

In [26]:
def calculate_hit_rate(df, strategy_daily_return_col):
    
    # Identify profitable trades (daily returns > 0)
    df['profitable_trade'] = df[strategy_daily_return_col] > 0

    # Calculate hit rate
    total_trades = df['profitable_trade'].count()
    profitable_trades = df['profitable_trade'].sum()
    hit_rate = profitable_trades / total_trades
    
    return hit_rate

In [44]:
def calculate_risk_and_performance_metrics(df, strategy_daily_return_col, annual_rf=0.05, annual_trading_days=252):
    
    # Calculate CAGR
    annualized_return = calculate_CAGR(df, strategy_daily_return_col=strategy_daily_return_col, annual_trading_days=annual_trading_days)
    
    # Calculate Annualized Sharpe Ratio
    annualized_sharpe_ratio = sharpe_ratio(df, daily_return_col=strategy_daily_return_col, annual_trading_days=annual_trading_days, annual_rf=annual_rf)
    
    # Calculate Annualized Standard Deviation
    annualized_std_dev = df[strategy_daily_return_col].std() * np.sqrt(annual_trading_days)
    
    # Calculate Max Drawdown
    df, max_drawdown, max_drawdown_duration = calculate_drawdown(df, strategy_daily_return_col=strategy_daily_return_col)
    
    # Calculate Hit Rate
    hit_rate = calculate_hit_rate(df, strategy_daily_return_col=strategy_daily_return_col)
    
    performance_metrics = {'annualized_return': annualized_return,
                           'annualized_sharpe_ratio': annualized_sharpe_ratio,
                           'annualized_std_dev': annualized_std_dev,
                           'max_drawdown': max_drawdown,
                           'max_drawdown_duration': max_drawdown_duration,
                           'hit_rate': hit_rate}
    
    return performance_metrics

In [98]:
data = get_close_prices(start_date, end_date, ticker)
data = get_long_and_short_term_momentum(data, close_px_col='ETH-USD', returns_type='logarithmic', weekend_trading=True)
data['ETH-USD_12_month_minus_1_month_momentum'] = data['ETH-USD_12_month_momentum'] - data['ETH-USD_1_month_momentum']

File data found...reading ['ETH-USD'] data


In [107]:
ticker_list = ['BCH-USD','ETH-USD','XRP-USD','SOL-USD','BTC-USD']
momentum_strategy_list = ['10_day_momentum','1_month_momentum','3_month_momentum','6_month_momentum','12_month_momentum','12_month_minus_1_month_momentum']
threshold_list = [0.2] * len(momentum_strategy_list)
performance_columns = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate']
df_results = pd.DataFrame(columns=performance_columns)
for ticker in ticker_list:
    print(ticker)
    data = get_close_prices(start_date, end_date, ticker)
    data = get_long_and_short_term_momentum(data, close_px_col=ticker, returns_type='logarithmic', weekend_trading=True)
    data[f'{ticker}_12_month_minus_1_month_momentum'] = data[f'{ticker}_12_month_momentum'] - data[f'{ticker}_1_month_momentum']
    for i in np.arange(len(momentum_strategy_list)):
        data = generate_momentum_signal(data, ticker=ticker, momentum_type=momentum_strategy_list[i], threshold=threshold_list[i])
        data = data.dropna()
        metrics = calculate_risk_and_performance_metrics(data, strategy_daily_return_col=f'{ticker}_{momentum_strategy_list[i]}_strategy_returns')
        df_results.loc[f'{ticker}_{momentum_strategy_list[i]}_strategy_returns'] = metrics

BCH-USD
File data found...reading ['BCH-USD'] data
ETH-USD
File data found...reading ['ETH-USD'] data
XRP-USD
File data found...reading ['XRP-USD'] data
SOL-USD
File not found...downloading the ['SOL-USD'] data
[*********************100%***********************]  1 of 1 completed
BTC-USD
File data found...reading ['BTC-USD'] data


In [64]:
# strategy_return_cols = ['BTC-USD_10_day_momentum_strategy_returns','BTC-USD_1_month_momentum_strategy_returns','BTC-USD_3_month_momentum_strategy_returns',
#                         'BTC-USD_6_month_momentum_strategy_returns','BTC-USD_12_month_momentum_strategy_returns','BTC-USD_12_month_minus_1_month_momentum_strategy_returns']
# performance_columns = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate']
# df_results = pd.DataFrame(columns=performance_columns)
# for strategy in strategy_return_cols:
#     metrics = calculate_risk_and_performance_metrics(data, strategy_daily_return_col=strategy)
#     df_results.loc[strategy] = metrics

In [96]:
data.head()

,ETH-USD,ETH-USD_log_returns,ETH-USD_10_day_momentum,ETH-USD_1_month_momentum,ETH-USD_3_month_momentum,ETH-USD_6_month_momentum,ETH-USD_12_month_momentum,ETH-USD_12_month_minus_1_month_momentum,ETH-USD_10_day_momentum_signal,ETH-USD_daily_returns,ETH-USD_10_day_momentum_strategy_returns,ETH-USD_10_day_momentum_cum_strategy_returns,ETH-USD_10_day_momentum_cum_buy_and_hold_returns,strategy_cumulative_return,strategy_cumulative_return_cum_max,strategy_cumulative_return_drawdown,strategy_cumulative_return_drawdown_pct,End,Start,strategy_cumulative_return_DDDuration,profitable_trade,ETH-USD_1_month_momentum_signal,ETH-USD_1_month_momentum_strategy_returns,ETH-USD_1_month_momentum_cum_strategy_returns,ETH-USD_1_month_momentum_cum_buy_and_hold_returns,ETH-USD_3_month_momentum_signal,ETH-USD_3_month_momentum_strategy_returns,ETH-USD_3_month_momentum_cum_strategy_returns,ETH-USD_3_month_momentum_cum_buy_and_hold_returns,ETH-USD_6_month_momentum_signal,ETH-USD_6_month_momentum_strategy_returns,ETH-USD_6_month_momentum_cum_strategy_returns,ETH-USD_6_month_momentum_cum_buy_and_hold_returns,ETH-USD_12_month_momentum_signal,ETH-USD_12_month_momentum_strategy_returns,ETH-USD_12_month_momentum_cum_strategy_returns,ETH-USD_12_month_momentum_cum_buy_and_hold_returns,ETH-USD_12_month_minus_1_month_momentum_signal,ETH-USD_12_month_minus_1_month_momentum_strategy_returns,ETH-USD_12_month_minus_1_month_momentum_cum_strategy_returns,ETH-USD_12_month_minus_1_month_momentum_cum_buy_and_hold_returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-05-14,2056.273926,0.020565,-0.357733,-0.356872,-0.224858,-0.407914,-0.636955,-0.280083,-1.0,-0.151353,-0.020778,-0.170346,1.815396,-0.151353,-0.151353,0.000000,-0.000000,2022-05-14,1,32 days,False,-1.0,-0.020778,-0.183042,-0.328512,-1.0,-0.020778,0.023697,-0.279295,-1.0,-0.020778,-0.320681,-0.423004,-1.0,-0.020778,0.023019,-0.188859,1.0,-0.151353,-0.151353,-0.157386
2022-05-15,2145.706787,0.042573,-0.247846,-0.309050,-0.160371,-0.386391,-0.610787,-0.301737,-1.0,0.043493,-0.043493,-0.206430,1.937845,-0.188262,-0.151353,-0.036910,0.243867,2022-05-15,1,32 days,False,-1.0,-0.043493,-0.218573,-0.299307,-1.0,-0.043493,-0.020827,-0.247950,-1.0,-0.043493,-0.350227,-0.397909,-1.0,-0.043493,-0.021474,-0.153580,-1.0,-0.043493,-0.188262,-0.120739
2022-05-17,2090.409180,0.032914,-0.231938,-0.295219,-0.225872,-0.437630,-0.494149,-0.198930,-1.0,-0.025771,-0.033461,-0.189022,1.862133,-0.167343,-0.151353,-0.015990,0.105650,2022-05-17,1,32 days,True,-1.0,-0.033461,-0.201432,-0.317365,-1.0,-0.033461,-0.053591,-0.267331,-1.0,0.025771,-0.333481,-0.413426,-1.0,0.025771,0.003743,-0.175394,-1.0,0.025771,-0.167343,-0.143399
2022-06-14,1211.662842,0.005860,-0.396687,-0.490589,-0.916693,-0.946675,-1.066046,-0.575457,-1.0,0.005878,-0.005878,-0.035421,0.658977,-0.172237,-0.151353,-0.020884,0.137986,2022-06-14,1,32 days,False,-1.0,-0.005878,0.142531,-0.604325,-1.0,-0.005878,0.354051,-0.575324,-1.0,-0.005878,-0.046395,-0.660004,-1.0,-0.005878,0.436081,-0.522034,-1.0,-0.005878,-0.172237,-0.503489
2022-06-15,1233.206421,0.017624,-0.381057,-0.455556,-0.927685,-0.965788,-1.065902,-0.610346,-1.0,0.017780,-0.017780,-0.052571,0.688474,-0.186955,-0.151353,-0.035602,0.235227,2022-06-15,1,32 days,False,-1.0,-0.017780,0.122217,-0.597290,-1.0,-0.017780,0.329976,-0.567773,-1.0,-0.017780,-0.063350,-0.653959,-1.0,-0.017780,0.410547,-0.513536,-1.0,-0.017780,-0.186955,-0.494660


In [108]:
df_results

,annualized_return,annualized_sharpe_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate
BCH-USD_10_day_momentum_strategy_returns,-0.079303,0.388961,1.089958,-17.902382,650 days,0.438017
BCH-USD_1_month_momentum_strategy_returns,-0.323476,0.622265,1.762136,-3.228571,514 days,0.494624
BCH-USD_3_month_momentum_strategy_returns,2.308462,1.435341,3.056012,-1.594908,514 days,0.481481
BCH-USD_6_month_momentum_strategy_returns,30.284620,2.182197,3.147680,-2.838183,375 days,0.500000
BCH-USD_12_month_momentum_strategy_returns,-0.775499,1.332066,3.506505,-1.596834,761 days,0.500000
BCH-USD_12_month_minus_1_month_momentum_strategy_returns,-0.970983,1.096250,4.001859,-5.534085,761 days,0.472727
ETH-USD_10_day_momentum_strategy_returns,-0.780337,-1.541369,0.802709,-3.982855,712 days,0.416667
ETH-USD_1_month_momentum_strategy_returns,-0.549729,0.449494,1.816222,-4.847961,712 days,0.436364
ETH-USD_3_month_momentum_strategy_returns,-0.947314,-0.353451,2.090290,-1.826686,704 days,0.431818
ETH-USD_6_month_momentum_strategy_returns,21.368278,2.131679,3.953129,-4.552681,153 days,0.500000


In [92]:
df_results

,annualized_return,annualized_sharpe_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate
BCH-USD_10_day_momentum_strategy_returns,-0.079303,0.388961,1.089958,-17.902382,650 days,0.438017
BCH-USD_1_month_momentum_strategy_returns,-0.323476,0.622265,1.762136,-3.228571,514 days,0.494624
BCH-USD_3_month_momentum_strategy_returns,2.308462,1.435341,3.056012,-1.594908,514 days,0.481481
BCH-USD_6_month_momentum_strategy_returns,30.284620,2.182197,3.147680,-2.838183,375 days,0.500000
BCH-USD_12_month_momentum_strategy_returns,-0.775499,1.332066,3.506505,-1.596834,761 days,0.500000
BCH-USD_12_month_minus_1_month_momentum_strategy_returns,-0.970983,1.096250,4.001859,-5.534085,761 days,0.472727
ETH-USD_10_day_momentum_strategy_returns,-0.780337,-1.541369,0.802709,-3.982855,712 days,0.416667
ETH-USD_1_month_momentum_strategy_returns,-0.549729,0.449494,1.816222,-4.847961,712 days,0.436364
ETH-USD_3_month_momentum_strategy_returns,-0.947314,-0.353451,2.090290,-1.826686,704 days,0.431818
ETH-USD_6_month_momentum_strategy_returns,21.368278,2.131679,3.953129,-4.552681,153 days,0.500000


In [101]:
data.head()

,ETH-USD,ETH-USD_log_returns,ETH-USD_10_day_momentum,ETH-USD_1_month_momentum,ETH-USD_3_month_momentum,ETH-USD_6_month_momentum,ETH-USD_12_month_momentum,ETH-USD_12_month_minus_1_month_momentum
Date,,,,,,,,
2021-01-01,730.367554,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02,774.534973,0.058715,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-03,975.507690,0.230695,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-04,1040.233032,0.064242,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,1100.006104,0.055871,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
calculate_CAGR(data, strategy_daily_return_col='ETH-USD_12_month_minus_1_month_momentum_strategy_returns')

-0.03341495280400264

In [59]:
data['strategy_cumulative_return'] = (1 + data['BTC-USD_10_day_momentum_strategy_returns']).cumprod() - 1

In [69]:
data = data.dropna()

In [70]:
data.head()

,BCH-USD,BCH-USD_log_returns,BCH-USD_10_day_momentum,BCH-USD_1_month_momentum,BCH-USD_3_month_momentum,BCH-USD_6_month_momentum,BCH-USD_12_month_momentum,BCH-USD_12_month_minus_1_month_momentum,BCH-USD_10_day_momentum_signal,BCH-USD_daily_returns,BCH-USD_10_day_momentum_strategy_returns,BCH-USD_10_day_momentum_cum_strategy_returns,BCH-USD_10_day_momentum_cum_buy_and_hold_returns,strategy_cumulative_return,strategy_cumulative_return_cum_max,strategy_cumulative_return_drawdown,strategy_cumulative_return_drawdown_pct,End,Start,strategy_cumulative_return_DDDuration,profitable_trade,BCH-USD_1_month_momentum_signal,BCH-USD_1_month_momentum_strategy_returns,BCH-USD_1_month_momentum_cum_strategy_returns,BCH-USD_1_month_momentum_cum_buy_and_hold_returns,BCH-USD_3_month_momentum_signal,BCH-USD_3_month_momentum_strategy_returns,BCH-USD_3_month_momentum_cum_strategy_returns,BCH-USD_3_month_momentum_cum_buy_and_hold_returns,BCH-USD_6_month_momentum_signal,BCH-USD_6_month_momentum_strategy_returns,BCH-USD_6_month_momentum_cum_strategy_returns,BCH-USD_6_month_momentum_cum_buy_and_hold_returns,BCH-USD_12_month_momentum_signal,BCH-USD_12_month_momentum_strategy_returns,BCH-USD_12_month_momentum_cum_strategy_returns,BCH-USD_12_month_momentum_cum_buy_and_hold_returns,BCH-USD_12_month_minus_1_month_momentum_signal,BCH-USD_12_month_minus_1_month_momentum_strategy_returns,BCH-USD_12_month_minus_1_month_momentum_cum_strategy_returns,BCH-USD_12_month_minus_1_month_momentum_cum_buy_and_hold_returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-23,301.807953,0.034952,-0.224005,-0.394629,-0.646190,-0.702493,-1.353881,-0.959253,-1.0,0.035570,-0.035570,-0.199617,-0.520498,-0.199617,0.501336,-0.053402,-0.106519,2022-01-23,73,3 days,False,-1.0,-0.035570,0.025661,-0.520498,-1.0,-0.035570,-0.053922,-0.520498,-1.0,-0.035570,-0.095739,-0.520498,-1.0,-0.035570,0.437899,-0.520498,-1.0,-0.035570,0.447934,-0.520498
2022-01-24,292.229126,-0.032253,-0.278747,-0.397679,-0.645596,-0.615652,-1.296620,-0.898941,-1.0,-0.031738,0.031738,-0.174215,-0.535717,-0.174215,0.501336,-0.007447,-0.014855,2022-01-24,73,3 days,True,-1.0,0.031738,0.058214,-0.535717,-1.0,0.031738,-0.023896,-0.535717,-1.0,0.031738,-0.067039,-0.535717,-1.0,0.031738,0.483535,-0.535717,-1.0,0.031738,0.493889,-0.535717
2022-01-25,292.233826,0.000016,-0.291547,-0.380457,-0.670192,-0.545060,-1.317800,-0.937343,-1.0,0.000016,-0.000016,-0.174228,-0.535709,-0.174228,0.501336,-0.007471,-0.014903,2022-01-25,73,3 days,False,-1.0,-0.000016,0.058197,-0.535709,-1.0,-0.000016,-0.023911,-0.535709,-1.0,-0.000016,-0.067054,-0.535709,-1.0,-0.000016,0.483511,-0.535709,-1.0,-0.000016,0.493865,-0.535709
2022-01-26,289.951355,-0.007841,-0.293125,-0.325675,-0.656653,-0.635914,-0.891029,-0.565354,-1.0,-0.007810,0.007810,-0.167778,-0.539336,-0.167778,0.505533,0.000000,0.000000,2022-01-26,74,0 days,True,-1.0,0.007810,0.066462,-0.539336,-1.0,0.007810,-0.016288,-0.539336,-1.0,0.007810,-0.059768,-0.539336,-1.0,0.007810,0.495098,-0.539336,-1.0,0.007810,0.505533,-0.539336
2022-01-27,289.510284,-0.001522,-0.274881,-0.324871,-0.749647,-0.645907,-1.043063,-0.718193,-1.0,-0.001521,0.001521,-0.166513,-0.540036,-0.166513,0.507823,0.000000,0.000000,2022-01-27,75,3 days,True,-1.0,0.001521,0.068084,-0.540036,-1.0,0.001521,-0.014791,-0.540036,-1.0,0.001521,-0.058338,-0.540036,-1.0,0.001521,0.497372,-0.540036,-1.0,0.001521,0.507823,-0.540036


In [35]:
momentum_strategy_list = ['10_day_momentum','1_month_momentum','3_month_momentum','6_month_momentum','12_month_momentum','12_month_minus_1_month_momentum']
for i in np.arange(len(momentum_strategy_list)):
    print(i)

0
1
2
3
4
5


In [33]:
[0.2] * 5

[0.2, 0.2, 0.2, 0.2, 0.2]

In [32]:
data.head()

,BCH-USD,BNB-USD,BTC-USD,ETH-USD,USDT-USD,XRP-USD,BTC-USD_log_returns,BTC-USD_10_day_momentum,BTC-USD_1_month_momentum,BTC-USD_3_month_momentum,BTC-USD_6_month_momentum,BTC-USD_12_month_momentum,BTC-USD_12_month_minus_1_month_momentum
Date,,,,,,,,,,,,,
2021-01-01,341.985657,37.905010,29374.152344,730.367554,1.001916,0.237444,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02,354.599945,38.241592,32127.267578,774.534973,1.000600,0.221655,0.089590,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-03,421.789856,41.148979,32782.023438,975.507690,1.000514,0.225848,0.020175,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-04,407.936920,40.926353,31971.914062,1040.233032,1.000128,0.236677,-0.025022,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,419.768127,41.734600,33992.429688,1100.006104,1.002202,0.226750,0.061280,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
data = get_long_and_short_term_momentum(data, close_px_col='ETH-USD', returns_type='logarithmic', weekend_trading=True)
data['ETH-USD_12_month_minus_1_month_momentum'] = data['ETH-USD_12_month_momentum'] - data['ETH-USD_1_month_momentum']

In [103]:
data = generate_momentum_signal(data, ticker='ETH-USD', momentum_type='12_month_minus_1_month_momentum', threshold=0.2)

In [104]:
data = data.dropna()
data.head(20)

,ETH-USD,ETH-USD_log_returns,ETH-USD_10_day_momentum,ETH-USD_1_month_momentum,ETH-USD_3_month_momentum,ETH-USD_6_month_momentum,ETH-USD_12_month_momentum,ETH-USD_12_month_minus_1_month_momentum,ETH-USD_12_month_minus_1_month_momentum_signal,ETH-USD_daily_returns,ETH-USD_12_month_minus_1_month_momentum_strategy_returns,ETH-USD_12_month_minus_1_month_momentum_cum_strategy_returns,ETH-USD_12_month_minus_1_month_momentum_cum_buy_and_hold_returns
Date,,,,,,,,,,,,,
2021-09-11,3270.278076,0.018135,-0.159250,0.013606,0.437522,-0.176780,1.440367,1.426761,1.0,0.018301,0.018301,0.018301,3.477579
2021-09-12,3410.134521,0.041877,-0.105875,0.050526,0.466103,-0.141594,1.251549,1.201023,1.0,0.042766,0.042766,0.061849,3.669066
2021-09-13,3285.511719,-0.037229,-0.181814,-0.010219,0.478181,-0.184774,1.150078,1.160296,1.0,-0.036545,-0.036545,0.023044,3.498436
2021-09-14,3429.169678,0.042796,-0.125533,0.077812,0.569587,-0.195286,1.137002,1.059190,1.0,0.043725,0.043725,0.067776,3.695129
2021-09-15,3615.282715,0.052852,-0.089086,0.114263,0.594861,-0.046100,1.096939,0.982676,1.0,0.054273,0.054273,0.125728,3.949950
2021-09-16,3571.294922,-0.012242,-0.095299,0.141421,0.625210,-0.039490,1.069434,0.928013,1.0,-0.012167,-0.012167,0.112031,3.889723
2021-09-17,3398.538818,-0.049583,-0.008163,0.038372,0.591870,-0.182520,1.021060,0.982688,1.0,-0.048374,-0.048374,0.058238,3.653190
2021-09-18,3432.018311,0.009803,-0.018847,0.056217,0.591913,-0.058319,0.985447,0.929230,1.0,0.009851,0.009851,0.068663,3.699029
2021-09-19,3329.447998,-0.030342,-0.028978,0.031253,0.563296,-0.074651,0.969913,0.938661,1.0,-0.029886,-0.029886,0.036725,3.558592


In [14]:
sharpe_ratio(data, daily_return_col='BTC-USD_12_month_minus_1_month_momentum_strategy_returns', annual_rf=0.06)

0.988914726715145

In [17]:
calculate_CAGR(data, strategy_daily_return_col='BTC-USD_12_month_minus_1_month_momentum_strategy_returns')

0.5044514669366058

In [27]:
calculate_hit_rate(data, strategy_daily_return_col='BTC-USD_12_month_minus_1_month_momentum_strategy_returns')

0.5240572171651495

In [ ]:
data['BTC-USD_12_month_minus_1_month_momentum_strategy_returns'].std()*np.sqrt(252)

In [22]:
data, max_drawdown, max_drawdown_duration = calculate_drawdown(data, strategy_daily_return_col='BTC-USD_12_month_minus_1_month_momentum_strategy_returns')

In [23]:
print(max_drawdown)
print(max_drawdown_duration)

-2.1358700697612423
390 days 00:00:00


In [ ]:
data['BTC-USD_12_month_minus_1_month_momentum_cum_strategy_returns_drawdown_pct'].min()

In [ ]:
# Calculate the total cumulative return
total_cumulative_return = data['BTC-USD_12_month_minus_1_month_momentum_cum_strategy_returns'].iloc[-1]

# Calculate the number of periods (days)
num_periods = len(data)

# Convert the number of periods to years (assuming daily data, 252 trading days per year)
num_years = num_periods / 252

# Calculate the annualized return
annualized_return = (1 + total_cumulative_return) ** (1 / num_years) - 1

In [ ]:
annualized_return

In [ ]:
data

In [ ]:
print(max)

In [ ]:
data[f'12_month_minus_1_month_momentum_signal'] = np.where(data['BTC-USD_12_month_minus_1_month_momentum'] > 0.2, 1,
                                                           np.where(data['BTC-USD_12_month_minus_1_month_momentum'] < -0.2, -1, np.nan))

In [ ]:
data[f'12_month_minus_1_month_momentum_signal'].dropna().head()

In [ ]:
data[f'12_month_minus_1_month_momentum_signal'].dropna().shift(1).head()

In [ ]:
data['BTC-USD_log_returns'][-11:-1].sum()

In [ ]:
fig = plt.figure(figsize=(22,8))
# plt.style.use('bmh')
layout = (3,3)
momentum_ax_1 = plt.subplot2grid(layout, (0,0))#, colspan=2)
momentum_ax_2 = plt.subplot2grid(layout, (0,1))#, colspan=2)
momentum_ax_3 = plt.subplot2grid(layout, (0,2))#, colspan=2)
momentum_ax_4 = plt.subplot2grid(layout, (1,0))#, colspan=2)
momentum_ax_5 = plt.subplot2grid(layout, (1,1))#, colspan=2)
momentum_ax_6 = plt.subplot2grid(layout, (1,2))#, colspan=2)
# pnl_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)

momentum_twin_ax_1 = momentum_ax_1.twinx()
_ = momentum_ax_1.plot(data.index, data['BTC-USD_10_day_momentum'], label='10 Day')
_ = momentum_twin_ax_1.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_2 = momentum_ax_2.twinx()
_ = momentum_ax_2.plot(data.index, data['BTC-USD_1_month_momentum'], label='1 Month')
_ = momentum_twin_ax_2.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_3 = momentum_ax_3.twinx()
_ = momentum_ax_3.plot(data.index, data['BTC-USD_3_month_momentum'], label='3 Month')
_ = momentum_twin_ax_3.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_4 = momentum_ax_4.twinx()
_ = momentum_ax_4.plot(data.index, data['BTC-USD_6_month_momentum'], label='6 Month')
_ = momentum_twin_ax_4.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_5 = momentum_ax_5.twinx()
_ = momentum_ax_5.plot(data.index, data['BTC-USD_12_month_momentum'], label='12 Month')
_ = momentum_twin_ax_5.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_6 = momentum_ax_6.twinx()
_ = momentum_ax_6.plot(data.index, data['BTC-USD_12_month_minus_1_month_momentum'], label='12 Month Minus 1 Month')
_ = momentum_twin_ax_6.plot(data.index, data['BTC-USD'], label='Price', color='orange')

_ = momentum_ax_1.grid()
_ = momentum_ax_1.legend()
_ = momentum_ax_2.grid()
_ = momentum_ax_2.legend()
_ = momentum_ax_3.grid()
_ = momentum_ax_3.legend()
_ = momentum_ax_4.grid()
_ = momentum_ax_4.legend()
_ = momentum_ax_5.grid()
_ = momentum_ax_5.legend()
_ = momentum_ax_6.grid()
_ = momentum_ax_6.legend()

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(40,5))
# plt.style.use('bmh')
layout = (2,1)
momentum_ax_1 = plt.subplot2grid(layout, (0,0))#, colspan=2)
momentum_ax_2 = plt.subplot2grid(layout, (1,0))#, colspan=2)
# momentum_ax_3 = plt.subplot2grid(layout, (0,2))#, colspan=2)
# momentum_ax_4 = plt.subplot2grid(layout, (0,3))#, colspan=2)
# momentum_ax_5 = plt.subplot2grid(layout, (0,4))#, colspan=2)
# pnl_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)

momentum_twin_ax_1 = momentum_ax_6.twinx()
_ = momentum_ax_1.plot(data.index, data['BTC-USD_12_month_minus_1_month_momentum'], label='12 Month Minus 1 Month')
_ = momentum_twin_ax_1.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_2 = momentum_ax_2.twinx()
_ = momentum_ax_2.plot(data.index, data['BTC-USD_1_month_momentum'], label='1 Month')
_ = momentum_twin_ax_2.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_3 = momentum_ax_3.twinx()
_ = momentum_ax_3.plot(data.index, data['BTC-USD_3_month_momentum'], label='3 Month')
_ = momentum_twin_ax_3.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_4 = momentum_ax_4.twinx()
_ = momentum_ax_4.plot(data.index, data['BTC-USD_6_month_momentum'], label='6 Month')
_ = momentum_twin_ax_4.plot(data.index, data['BTC-USD'], label='Price', color='orange')

momentum_twin_ax_5 = momentum_ax_5.twinx()
_ = momentum_ax_5.plot(data.index, data['BTC-USD_12_month_momentum'], label='12 Month')
_ = momentum_twin_ax_5.plot(data.index, data['BTC-USD'], label='Price', color='orange')

_ = momentum_ax_1.grid()
_ = momentum_ax_1.legend()
_ = momentum_ax_2.grid()
_ = momentum_ax_2.legend()
_ = momentum_ax_3.grid()
_ = momentum_ax_3.legend()
_ = momentum_ax_4.grid()
_ = momentum_ax_4.legend()
_ = momentum_ax_5.grid()
_ = momentum_ax_5.legend()

plt.tight_layout()

In [ ]:
data.tail()

In [ ]:
def generate_trend_signal(df, mavg_slope_col_list, ticker='BTC-USD'):
    
    def slope_signal(row):
        if all(row[i] <= row[i+1] for i in range(len(row) - 1)):
            return 'downtrend'
        elif all(row[i] >= row[i+1] for i in range(len(row) - 1)):
            return 'uptrend'
        
    df[f'{ticker}_trend_slope_signal'] = df[mavg_slope_col_list].apply(slope_signal, axis=1)

    return df

In [ ]:
for end in np.linspace(200, 600, 3):
    print(end)

In [ ]:
for end in np.arange(200, 1000, 50):
    for start in np.arange(10, end, 10):
        for stepsizes in np.arange(1,10,1):
            print(start, end, end/start*stepsizes)

In [ ]:
# Specify date range and pull data for specific instrument
start_date = pd.datetime(2021, 1, 1)
end_date = pd.datetime(2024, 7, 25)
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
# ticker = ['TSLA']
data = get_close_prices(start_date, end_date, ticker)

In [ ]:
df_tsla = get_close_prices(start_date, end_date, ticker='TSLA')

In [ ]:
df_tsla.head()

In [ ]:
len(ticker)

In [ ]:
data.head(10)

In [ ]:
ticker = ['SPY']
data_SPY = get_close_prices(start_date, end_date, ticker=ticker)

In [ ]:
data_SPY.head(10)

In [ ]:
# Specify date range and pull data for specific instrument
start_date = pd.datetime(2015, 1, 1)
end_date = pd.datetime(2024, 7, 21)
ticker = ['PL=F']
data = get_close_prices(start_date, end_date, ticker=ticker)

In [ ]:
data.head()

In [ ]:
# def create_trend_strategy(df, ticker, mavg_start, mavg_end, mavg_stepsize, vol_range_list=[10, 20, 30, 60, 90]):
    
#     for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
#         df[f'{ticker}_{int(window)}_mavg'] = df[f'{ticker}'].rolling(int(window)).mean()
#         df[f'{ticker}_{int(window)}_mavg_slope'] = calculate_slope(df, column=f'{ticker}_{int(window)}_mavg', periods=window)

#     df[f'{ticker}_ribbon_thickness'] = df[f'{ticker}_{int(mavg_start)}_mavg'] - df[f'{ticker}_{int(mavg_end)}_mavg']
#     df[f'{ticker}_ribbon_thickness_diff'] = df[f'{ticker}_ribbon_thickness'].diff()
#     df = get_returns_volatility(df, vol_range_list=vol_range_list, close_px_col=ticker)

#     ## Ticker Trend Signal and Trade
#     mavg_col_list = [f'{ticker}_{int(mavg)}_mavg' for mavg in np.linspace(mavg_start, mavg_end, mavg_stepsize).tolist()]
#     mavg_slope_col_list = [f'{ticker}_{int(mavg)}_mavg_slope' for mavg in np.linspace(mavg_start, mavg_end, mavg_stepsize).tolist()]
#     df[f'{ticker}_trend_signal'] = df[mavg_col_list].apply(trend_signal, axis=1)
#     df[f'{ticker}_trend_signal_diff'] = df[f'{ticker}_trend_signal'].diff().shift(1)
#     df[f'{ticker}_trend_trade'] = np.where(df[f'{ticker}_trend_signal_diff'] != 0, df[f'{ticker}'], np.nan)
#     df[f'{ticker}_trend_strategy_returns'] = df[f'{ticker}_pct_returns'] * df[f'{ticker}_trend_signal_diff']
    
#     ## Ticker Trend Slope Signal and Trade
#     df[f'{ticker}_trend_slope_signal'] = df[mavg_slope_col_list].apply(slope_signal, axis=1)
#     df[f'{ticker}_trend_slope_signal_diff'] = df[f'{ticker}_trend_slope_signal'].diff().shift(1)
#     df[f'{ticker}_trend_slope_trade'] = np.where(df[f'{ticker}_trend_slope_signal_diff'] != 0, df[f'{ticker}'], np.nan)
#     df[f'{ticker}_trend_slope_strategy_returns'] = df[f'{ticker}_pct_returns'] * df[f'{ticker}_trend_slope_signal_diff']
    
#     ## Drop all null values
#     df = df[df[f'{ticker}_{mavg_end}_mavg_slope'].notnull()]
    
#     ## Calculate P&L
#     df[f'{ticker}_mavg_trend_PnL'] = df[f'{ticker}_trend_signal_diff'] * df[f'{ticker}_trend_trade'] * -1
#     df[f'{ticker}_mavg_slope_PnL'] = df[f'{ticker}_trend_slope_signal_diff'] * df[f'{ticker}_trend_slope_trade'] * -1
    
#     ## Calculate Cumulative P&L
#     df[f'{ticker}_mavg_trend_PnL_cum'] = df[f'{ticker}_mavg_trend_PnL'].cumsum()
#     df[f'{ticker}_mavg_slope_PnL_cum'] = df[f'{ticker}_mavg_slope_PnL'].cumsum()
   
#     return df

In [ ]:
mavg_list = []
_mavg = []
mavg_start_list = [mavg_start for mavg_start in np.arange(10,110,10)]
mavg_end_list = [mavg_end for mavg_end in np.arange(100,1100,100)]
for i in np.arange(0,10,1):
    mavg_list.append([mavg_start_list[i], mavg_end_list[i], 2])
    mavg_list.append([mavg_start_list[i], mavg_end_list[i], 4])
    mavg_list.append([mavg_start_list[i], mavg_end_list[i], 10])
    mavg_list.append([mavg_start_list[i], mavg_end_list[i], int(mavg_start_list[i] - mavg_start_list[i]/10+1)])
#     mavg_list.append(_mavg)
#     print(i, mavg_start_list[i], mavg_end_list[i], )

In [ ]:
mavg_list

In [ ]:
def get_mavg_windows():
    mavg_list = []
    _mavg = []
    mavg_start_list = [mavg_start for mavg_start in np.arange(10,110,10)]
    mavg_end_list = [mavg_end for mavg_end in np.arange(100,1100,100)]
    for i in np.arange(0,10,1):
        mavg_list.append([mavg_start_list[i], mavg_end_list[i], 2])
        mavg_list.append([mavg_start_list[i], mavg_end_list[i], 4])
        mavg_list.append([mavg_start_list[i], mavg_end_list[i], 10])
        mavg_list.append([mavg_start_list[i], mavg_end_list[i], int(mavg_start_list[i] - mavg_start_list[i]/10+1)])
    
    return mavg_list

In [ ]:
def get_strategy_performance(start_date, end_date, ticker_list):
    if isinstance(ticker_list, str):
        ticker_list = [ticker_list]
    mavg_list = get_mavg_windows()
    all_results = []
    for ticker in ticker_list:
        print(ticker)
        _results = []
        for mavg_start, mavg_end, mavg_stepsize in mavg_list:
            df = get_close_prices(start_date, end_date, ticker)
            df = create_trend_strategy(df, ticker, mavg_start, mavg_end, mavg_stepsize)
            _sharpe = sharpe_ratio(df, return_col=f'{ticker}_trend_strategy_returns', trade_col=f'{ticker}_trend_trade')
            _pnl = df[f'{ticker}_mavg_trend_PnL'].sum()
            _results.append([ticker, mavg_start, mavg_end, mavg_stepsize, round(_sharpe, 2), round(_pnl, 2)])
        all_results.append(_results)
    flattened_list = [item for sublist in all_results for item in sublist]
    
    columns = ['ticker','mavg_start','mavg_end','mavg_stepsize','sharpe_ratio','pnl']
    df_results = pd.DataFrame(flattened_list, columns=columns)
    
    return df_results

In [ ]:
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
df_results = get_strategy_performance(start_date, end_date, ticker)

In [ ]:
fig = plt.figure(figsize=(15,8))
# plt.style.use('bmh')
layout = (1,2)
sharpe_ax = plt.subplot2grid(layout, (0,0))#, colspan=2)
pnl_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)

mavg_cond = (df_results.mavg_start == 10) & (df_results.mavg_end == 100) & (df_results.mavg_stepsize == 4)
_ = sharpe_ax.plot(df_results[mavg_cond].ticker, df_results[mavg_cond]['sharpe_ratio'])
_ = sharpe_ax.grid()
_ = pnl_ax.plot(df_results[mavg_cond].ticker, df_results[mavg_cond]['pnl'])
_ = pnl_ax.grid()

plt.tight_layout()

In [ ]:
df_results[mavg_cond]

In [ ]:
## Simple Moving Average Results
results = []
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='BTC-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize)
    _sharpe = sharpe_ratio(data, return_col='BTC-USD_trend_strategy_returns', trade_col='BTC-USD_trend_trade')
    _pnl = data[f'BTC-USD_mavg_trend_PnL'].sum()
    results.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results

In [ ]:
## Simple Moving Average Results
results = []
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='BTC-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize, moving_avg_type='ewm')
    _sharpe = sharpe_ratio(data, return_col='BTC-USD_trend_strategy_returns', trade_col='BTC-USD_trend_trade')
    _pnl = data[f'BTC-USD_mavg_trend_PnL'].sum()
    results.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results

In [ ]:
## Simple Moving Average Results
results_ETH = []
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='ETH-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize)
    _sharpe = sharpe_ratio(data, return_col='ETH-USD_trend_strategy_returns', trade_col='ETH-USD_trend_trade')
    _pnl = data[f'ETH-USD_mavg_trend_PnL'].sum()
    results_ETH.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results_ETH

In [ ]:
## Simple Moving Average Results
results_ETH = []
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='ETH-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize, moving_avg_type='ewm')
    _sharpe = sharpe_ratio(data, return_col='ETH-USD_trend_strategy_returns', trade_col='ETH-USD_trend_trade')
    _pnl = data[f'ETH-USD_mavg_trend_PnL'].sum()
    results_ETH.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results_ETH

In [ ]:
## Simple Moving Average Results
results = []
ticker = ['PL=F','ES=F']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='ES=F', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize)
    _sharpe = sharpe_ratio(data, return_col='ES=F_trend_strategy_returns', trade_col='ES=F_trend_trade')
    _pnl = data[f'ES=F_mavg_trend_PnL'].sum()
    results.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results

In [ ]:
ticker = ['PL=F','ES=F']
data = load_financial_data(start_date, end_date, ticker=ticker)
data = data['Adj Close']
# data = create_trend_stra.tegy(data, ticker='PL=F', mavg_start=10, mavg_end=100, mavg_stepsize=4)

In [ ]:
fig = plt.figure(figsize=(15,8))
# plt.style.use('bmh')
layout = (1,2)
pl_prices_ax = plt.subplot2grid(layout, (0,0))#, colspan=2)
es_prices_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)

_ = pl_prices_ax.plot(data.index, data['PL=F'])
_ = es_prices_ax.plot(data.index, data['ES=F'])

plt.tight_layout()

In [ ]:
mavg_end

In [ ]:
data.tail(500)

In [ ]:
ticker = 'ES=F'
moving_avg_type='simple'
for window in np.linspace(10, 100, 4):
    if moving_avg_type == 'simple':
        data[f'{ticker}_{int(window)}_mavg'] = data[f'{ticker}'].rolling(int(window)).mean()
    else:
        data[f'{ticker}_{int(window)}_mavg'] = data[f'{ticker}'].ewm(span=window).mean()
    data[f'{ticker}_{int(window)}_mavg_slope'] = calculate_slope(data, column=f'{ticker}_{int(window)}_mavg',
                                                               periods=window)

In [ ]:
data.tail(500)

In [ ]:
results

In [ ]:
## Simple Moving Average Results
results = []
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data['Adj Close']
    data = create_trend_strategy(data, ticker='BTC-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize, moving_avg_type='ewm')
    _sharpe = sharpe_ratio(data, return_col='BTC-USD_trend_strategy_returns', trade_col='BTC-USD_trend_trade')
    _pnl = data[f'BTC-USD_mavg_trend_PnL'].sum()
    results.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results

In [ ]:
data = load_financial_data(start_date, end_date, ticker=ticker)

In [ ]:
data.head()

In [ ]:
results_SPY = []
ticker = ['SPY']
for mavg_start, mavg_end, mavg_stepsize in mavg_list:
    print(mavg_start, mavg_end, mavg_stepsize)
    data = load_financial_data(start_date, end_date, ticker=ticker)
    data = data[['Adj Close']].rename(columns={'Adj Close': 'SPY'})
    data = create_trend_strategy(data, ticker='SPY', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize)
    _sharpe = sharpe_ratio(data, return_col='SPY_trend_strategy_returns', trade_col='SPY_trend_trade')
    _pnl = data[f'SPY_mavg_trend_PnL'].sum()
    results_SPY.append([mavg_start, mavg_end, mavg_stepsize, round(_sharpe,2), round(_pnl,2)])

In [ ]:
results_SPY

In [ ]:
mavg_start = 10
mavg_end = 100
mavg_stepsize = 4

In [ ]:
ticker = ['BCH-USD','ETH-USD','USDT-USD','XRP-USD','BNB-USD','BTC-USD']
# ticker = ['PL=F','ES=F']
data = load_financial_data(start_date, end_date, ticker=ticker)
data = data['Adj Close']
data = create_trend_strategy(data, ticker='BTC-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize, moving_avg_type='ewm')

In [ ]:
sharpe_ratio(data, return_col='BTC-USD_trend_strategy_returns', trade_col='BTC-USD_trend_trade')

In [ ]:
data['BTC-USD_mavg_trend_PnL'].sum()

In [ ]:
data.head()

In [ ]:
data = create_trend_strategy(data, ticker='ETH-USD', mavg_start=mavg_start, mavg_end=mavg_end, mavg_stepsize=mavg_stepsize)

In [ ]:
sharpe_ratio(data, return_col='ETH-USD_trend_strategy_returns', trade_col='ETH-USD_trend_trade')

In [ ]:
data[data['ETH-USD_trend_trade'].notnull()]

In [ ]:
mavg_start_SPY = 50
mavg_end_SPY = 200
mavg_stepsize_SPY = 2

In [ ]:
for window in np.linspace(mavg_start_SPY, mavg_end_SPY, mavg_stepsize_SPY):
    print(window)
    data_SPY[f'SPY_{window}_mavg'] = data_SPY['SPY'].rolling(int(window)).mean()
#     data_SPY[f'SPY_{window}_mavg_slope'] = (data_SPY[f'SPY_10.0_mavg'] - data_SPY[f'SPY_10.0_mavg'].shift(int(window)))/window
    data_SPY[f'SPY_{window}_mavg_slope'] = calculate_slope(data_SPY, column=f'SPY_{window}_mavg', periods=window)
    
data_SPY['SPY_ribbon_thickness'] = data_SPY[f'SPY_{float(mavg_start_SPY)}_mavg'] - data_SPY[f'SPY_{float(mavg_end_SPY)}_mavg']
data_SPY = get_returns_volatility(data_SPY, vol_range_list=[10, 20, 30, 60, 90], close_px_col='SPY')

## SPY Trend Signal and Trade
SPY_mavg_col_list = [f'SPY_{mavg}_mavg' for mavg in np.linspace(mavg_start_SPY, mavg_end_SPY, mavg_stepsize_SPY).tolist()]
SPY_mavg_slope_col_list = [f'SPY_{mavg}_mavg_slope' for mavg in np.linspace(mavg_start_SPY, mavg_end_SPY, mavg_stepsize_SPY).tolist()]
data_SPY['SPY_trend_signal'] = data_SPY[SPY_mavg_col_list].apply(trend_signal, axis=1)
data_SPY['SPY_trend_signal_diff'] = data_SPY['SPY_trend_signal'].diff().shift(1)
data_SPY['SPY_trend_trade'] = np.where(data_SPY['SPY_trend_signal_diff'] != 0, data_SPY['SPY'], np.nan)
data_SPY['SPY_trend_slope_signal'] = data_SPY[SPY_mavg_slope_col_list].apply(slope_signal, axis=1)
data_SPY['SPY_trend_slope_signal_diff'] = data_SPY['SPY_trend_slope_signal'].diff().shift(1)
data_SPY['SPY_trend_slope_trade'] = np.where(data_SPY['SPY_trend_slope_signal_diff'] != 0, data_SPY['SPY'], np.nan)
data_SPY['SPY_trend_strategy_returns'] = data_SPY[f'SPY_pct_returns'] * data_SPY['SPY_trend_signal_diff']
data_SPY['SPY_trend_slope_strategy_returns'] = data_SPY[f'SPY_pct_returns'] * data_SPY['SPY_trend_slope_signal_diff']
# data_SPY['SPY_trend_trade'] = np.where((np.abs(data_SPY['SPY_ribbon_thickness']) > 2000) &
#                                    (data_SPY['SPY_trend_signal_diff'] != 0), data_SPY['SPY_trend_signal_diff'], np.nan)

In [ ]:
data.shape

In [ ]:
data_SPY.shape

In [ ]:
# data = data.dropna()
# data = data[data.index.get_level_values('Date') >= '2018-12-14']
data_SPY = data_SPY[data_SPY.index.get_level_values('Date') >= '2019-06-14']

In [ ]:
data['BTC_mavg_slope_PnL'] = data['BTC_trend_slope_signal_diff'] * data['BTC_trend_slope_trade'] * -1
BTC_cols = [col for col in data.columns if (col.startswith('BTC')) & (col[-5:] == 'slope')]
BTC_cols.append('BTC_trend_slope_signal')
BTC_cols.append('BTC_trend_slope_signal_diff')
BTC_cols.append('BTC_mavg_slope_PnL')
BTC_cols.append('BTC-USD_pct_returns')
BTC_cols.append('BTC_trend_slope_strategy_returns')
cond = (data['BTC_trend_slope_signal_diff'] != 0)
data[cond][BTC_cols]

In [ ]:
data['BTC_mavg_trend_PnL'] = data['BTC_trend_signal_diff'] * data['BTC_trend_trade'] * -1
BTC_cols = [col for col in data.columns if (col.startswith('BTC')) & (col[-5:] == 'slope')]
BTC_cols.append('BTC_trend_signal')
BTC_cols.append('BTC_trend_signal_diff')
BTC_cols.append('BTC_mavg_trend_PnL')
BTC_cols.append('BTC-USD_pct_returns')
BTC_cols.append('BTC_trend_strategy_returns')
cond = (data['BTC_trend_trade'].notnull())
data[cond][BTC_cols]

In [ ]:
data['ETH_mavg_slope_PnL'] = data['ETH_trend_slope_signal_diff'] * data['ETH_trend_slope_trade'] * -1
ETH_cols = [col for col in data.columns if (col.startswith('ETH')) & (col[-5:] == 'slope')]
ETH_cols.append('ETH_trend_slope_signal')
ETH_cols.append('ETH_trend_slope_signal_diff')
ETH_cols.append('ETH_mavg_slope_PnL')
cond = (data['ETH_trend_slope_trade'].notnull())
data[cond][ETH_cols]

In [ ]:
data['ETH_mavg_trend_PnL'] = data['ETH_trend_signal_diff'] * data['ETH_trend_trade'] * -1
ETH_cols = [col for col in data.columns if (col.startswith('ETH')) & (col[-5:] == 'slope')]
ETH_cols.append('ETH_trend_signal')
ETH_cols.append('ETH_trend_signal_diff')
ETH_cols.append('ETH_mavg_trend_PnL')
cond = (data['ETH_trend_trade'].notnull())
data[cond][ETH_cols]

In [ ]:
data_SPY['SPY_mavg_trend_PnL'] = data_SPY['SPY_trend_signal_diff'] * data_SPY['SPY_trend_trade'] * -1
SPY_cols = [col for col in data_SPY.columns if (col.startswith('SPY')) & (col[-5:] == 'slope')]
SPY_cols.append('SPY_trend_signal')
SPY_cols.append('SPY_trend_signal_diff')
SPY_cols.append('SPY_mavg_trend_PnL')
cond = (data_SPY['SPY_trend_trade'].notnull())
data_SPY[cond][SPY_cols]

In [ ]:
# Calculate the mean and standard deviation of the strategy returns
# mean_strategy_return = data['strategy_return'].mean()
# std_strategy_return = data['strategy_return'].std()

# Calculate annualized Sharpe Ratio (assuming 252 trading days in a year)
N = 255 #255 trading days in a year
rf =0.05 #1% risk free rate
cond = (data['BTC_trend_trade'].notnull())
sharpes = sharpe_ratio(data[cond]['BTC_trend_strategy_returns'], N, rf)

In [ ]:
data.head()

In [ ]:
print(f"BTC-USD_mavg_trend_PnL: {data['BTC-USD_mavg_trend_PnL'].sum()}")
cond = (data['BTC-USD_trend_signal_diff'] != 0)
print(f"BTC-USD Trend Sharpe Ratio: {sharpe_ratio(data, return_col='BTC-USD_trend_strategy_returns', trade_col='BTC-USD_trend_trade')}")
print(f"BTC-USD_mavg_slope_PnL: {data['BTC-USD_mavg_slope_PnL'].sum()}")
cond = (data['BTC-USD_trend_slope_signal_diff'] != 0)
print(f"BTC-USD Trend Slope Sharpe Ratio: {sharpe_ratio(data, return_col='BTC-USD_trend_slope_strategy_returns', trade_col='BTC-USD_trend_slope_trade')}")

In [ ]:
print(f"ETH-USD_mavg_trend_PnL: {data['ETH-USD_mavg_trend_PnL'].sum()}")
cond = (data['ETH-USD_trend_signal_diff'] != 0)
print(f"ETH-USD Trend Sharpe Ratio: {sharpe_ratio(data, return_col='ETH-USD_trend_strategy_returns', trade_col='ETH-USD_trend_trade')}")
print(f"ETH-USD_mavg_slope_PnL: {data['ETH-USD_mavg_slope_PnL'].sum()}")
cond = (data['ETH-USD_trend_slope_signal_diff'] != 0)
print(f"ETH-USD Trend Slope Sharpe Ratio: {sharpe_ratio(data, return_col='ETH-USD_trend_slope_strategy_returns', trade_col='ETH-USD_trend_slope_trade')}")

In [ ]:
[col for col in data_SPY.columns if (col.startswith('SPY')) & (col[-4:] == 'mavg')]

In [ ]:
data_SPY.loc[data_SPY.index >= '2020-03-01'][['SPY','SPY_50.0_mavg','SPY_200.0_mavg','SPY_trend_signal','SPY_trend_signal_diff','SPY_mavg_trend_PnL','SPY_ribbon_thickness']].head(500)

In [ ]:
data.loc[data.index >= '2022-05-01'][['BTC-USD','BTC_20.0_mavg','BTC_40.0_mavg','BTC_60.0_mavg','BTC_80.0_mavg','BTC_100.0_mavg','BTC_120.0_mavg','BTC_140.0_mavg',
                                      'BTC_160.0_mavg','BTC_180.0_mavg','BTC_200.0_mavg','BTC_trend_signal','BTC_trend_signal_diff','BTC_mavg_trend_PnL','BTC_ribbon_thickness']].head(500)

In [ ]:
data.loc[data['BTC-USD_trend_trade'].notnull()][['BTC-USD','BTC-USD_20_mavg','BTC-USD_40_mavg','BTC-USD_60_mavg','BTC-USD_80_mavg','BTC-USD_100_mavg','BTC-USD_120_mavg','BTC-USD_140_mavg',
                                      'BTC-USD_160_mavg','BTC-USD_180_mavg','BTC-USD_200_mavg','BTC-USD_trend_signal','BTC-USD_trend_signal_diff','BTC-USD_mavg_trend_PnL','BTC-USD_ribbon_thickness']].head(500)

In [ ]:
data_Pnl = data.loc[data['BTC-USD_trend_trade'].notnull()][['BTC-USD','BTC-USD_20_mavg','BTC-USD_40_mavg','BTC-USD_60_mavg','BTC-USD_80_mavg','BTC-USD_100_mavg','BTC-USD_120_mavg','BTC-USD_140_mavg',
                                      'BTC-USD_160_mavg','BTC-USD_180_mavg','BTC-USD_200_mavg','BTC-USD_trend_signal','BTC-USD_trend_signal_diff','BTC-USD_mavg_trend_PnL','BTC-USD_ribbon_thickness']].head(500)

In [ ]:
data_Pnl['BTC-USD_mavg_trend_PnL_cum'] = data_Pnl['BTC-USD_mavg_trend_PnL'].cumsum()

In [ ]:
data[data['BTC-USD_trend_trade'].notnull()].head(100)

In [ ]:
data['BTC-USD_ribbon_thickness_diff'] = data['BTC-USD_ribbon_thickness'].diff()

In [ ]:
data.index[3-1]

In [ ]:
data.head(500)

In [ ]:
for j in range(1, 4):
    print(j)

In [ ]:
data['BTC-USD_ribbon_thickness_signal'] = np.nan
data.loc[data.index[0:3], 'BTC-USD_ribbon_thickness_signal'] = 0
lookback_period = 3
for i in range(lookback_period, len(data)):
    if all(data.loc[data.index[i-j], 'BTC-USD_ribbon_thickness'] > data.loc[data.index[i-j-1], 'BTC-USD_ribbon_thickness'] for j in range(0,lookback_period-1)):
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_signal'] = 1
    elif all(data.loc[data.index[i-j], 'BTC-USD_ribbon_thickness'] < data.loc[data.index[i-j-1], 'BTC-USD_ribbon_thickness'] for j in range(0,lookback_period-1)):
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_signal'] = -1
#     elif data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_signal'] == 'Buy' and data.loc[data.index[i], 'BTC-USD_ribbon_thickness_diff'] < 0:
#         data['BTC-USD_ribbon_thickness_signal'] = 'Exit Buy'
#     elif data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_signal'] == 'Sell' and data.loc[data.index[i], 'BTC-USD_ribbon_thickness_diff'] > 0:
#         data['BTC-USD_ribbon_thickness_signal'] = 'Exit Sell'
    else:
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_signal'] = 0#data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_signal']

data['BTC-USD_ribbon_thickness_position'] = data['BTC-USD_ribbon_thickness_signal'].diff()
        
data['BTC-USD_ribbon_thickness_PnL'] = data['BTC-USD_ribbon_thickness_position'] * data['BTC-USD']
data['BTC-USD_ribbon_thickness_cum_PnL'] = data['BTC-USD_ribbon_thickness_PnL'].cumsum()

In [ ]:
trend_buy_signal = (data['BTC-USD_trend_signal'] == 1)
trend_sell_signal = (data['BTC-USD_trend_signal'] == -1)
ribbon_thickness_buy_signal = (data['BTC-USD_ribbon_thickness_signal'] == 1)
ribbon_thickness_sell_signal = (data['BTC-USD_ribbon_thickness_signal'] == -1)
data['BTC-USD_trend_ribbon_thickness_combo_signal'] = np.where(trend_buy_signal & ribbon_thickness_buy_signal, 1,
                                                              np.where(trend_sell_signal & ribbon_thickness_sell_signal, -1, 0))
data['BTC-USD_trend_ribbon_thickness_combo_position'] = data['BTC-USD_trend_ribbon_thickness_combo_signal'].diff()
data['BTC-USD_trend_ribbon_thickness_combo_PnL'] = data['BTC-USD_trend_ribbon_thickness_combo_position'] * data['BTC-USD']
data['BTC-USD_trend_ribbon_thickness_combo_cum_PnL'] = data['BTC-USD_trend_ribbon_thickness_combo_PnL'].cumsum()

In [ ]:
data['BTC-USD_trend_ribbon_thickness_combo_PnL'].sum()

In [ ]:
data[['BTC-USD','BTC-USD_ribbon_thickness','BTC-USD_ribbon_thickness_diff','BTC-USD_ribbon_thickness_signal','BTC-USD_trend_signal','BTC-USD_ribbon_thickness_position','BTC-USD_ribbon_thickness_PnL',
     'BTC-USD_ribbon_thickness_cum_PnL']].head(500)

In [ ]:
data['BTC-USD_ribbon_thickness_PnL'].sum()

In [ ]:
data['BTC-USD_ribbon_thickness_slope'] = calculate_slope(data, column='BTC-USD_ribbon_thickness', periods=50)
data['BTC-USD_ribbon_thickness_slope_diff'] = data['BTC-USD_ribbon_thickness_slope'].diff()
data['BTC-USD_ribbon_thickness_slope_position'] = np.nan
data.loc[data.index[0], 'BTC-USD_ribbon_thickness_slope_position'] = 'No Position'
for i in range(3, len(data)):
    if all(data.loc[data.index[i-j], 'BTC-USD_ribbon_thickness_slope_diff'] > 0 for j in range(1,4)):
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_slope_position'] = 'Buy'
    elif all(data.loc[data.index[i-j], 'BTC-USD_ribbon_thickness_slope_diff'] < 0 for j in range(1,4)):
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_slope_position'] = 'Sell'
    elif data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_slope_position'] == 'Buy' and data.loc[data.index[i], 'BTC-USD_ribbon_thickness_diff'] < 0:
        data['BTC-USD_ribbon_thickness_position'] = 'Exit Buy'
    elif data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_position'] == 'Sell' and data.loc[data.index[i], 'BTC-USD_ribbon_thickness_diff'] > 0:
        data['BTC-USD_ribbon_thickness_position'] = 'Exit Sell'
    else:
        data.loc[data.index[i], 'BTC-USD_ribbon_thickness_position'] = data.loc[data.index[i-1], 'BTC-USD_ribbon_thickness_position']
        
        

In [ ]:
fig = plt.figure(figsize=(20,15))
# plt.style.use('bmh')
layout = (4,2)
mavg_ax = plt.subplot2grid(layout, (0,0), colspan=2)
ribbon_vol_ax = plt.subplot2grid(layout, (2,0), colspan=2)
signal_ax = plt.subplot2grid(layout, (1,0), colspan=2)
pnl_ax = plt.subplot2grid(layout, (3,0), colspan=2)
# trade_ax = plt.subplot2grid(layout, (1,1))

# _ = mavg_ax.plot(data.index, data['BTC-USD_short_mavg'], label='Short MAVG', linestyle='--', color='blue')
# _ = mavg_ax.plot(data.index, data['BTC-USD_long_mavg'], label='Long MAVG', linestyle='--', color='orange')
# _ = ribbon_ax.plot(data.index, data['BTC-USD_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='magenta', linewidth=2)
_ = mavg_ax.plot(data.index, data['BTC-USD'], label='Price', linestyle='--', color='magenta', linewidth=1, alpha=0.6)
for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
    _ = mavg_ax.plot(data.index, data[f'BTC-USD_{int(window)}_mavg'], label=f'{int(window)} MAVG', linestyle='--')
    
buy_cond = (data['BTC-USD_trend_signal_diff'] == 1)
sell_cond = (data['BTC-USD_trend_signal_diff'] == -1)
_ = mavg_ax.plot(data[buy_cond].index, data[buy_cond]['BTC-USD_trend_trade'], label='Buy', linestyle='', color='cyan', linewidth=50, marker='^')
_ = mavg_ax.plot(data[sell_cond].index, data[sell_cond]['BTC-USD_trend_trade'], label='Sell', linestyle='', color='red', linewidth=50, marker='v')
    
# _ = slope_ax.plot(data.index, data['BTC-USD'], label='Price', linestyle='--', color='magenta', linewidth=2, alpha=0.6)
# slope_ax2 = slope_ax.twinx()
# for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
#     _ = slope_ax2.plot(data.index, data[f'BTC-USD_{window}_mavg_slope'], label=f'{window} MAVG Slope', linestyle='--')

_ = signal_ax.plot(data.index, data['BTC-USD_trend_signal'], label='Trend Signal', linestyle='--', color='orange', linewidth=2)
# _ = signal_ax.plot(data.index, data['BTC-USD_trend_slope_signal'], label='Trend Slope Signal', linestyle='--', color='green', linewidth=2)#, marker='v')

_ = ribbon_vol_ax.plot(data.index, data['BTC-USD_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='red', linewidth=2)
ribbon_vol_ax2 = ribbon_vol_ax.twinx()
_ = ribbon_vol_ax2.plot(data.index, data['BTC-USD_volatility_10'], label='10 Day Vol', linestyle='--', linewidth=2)
_ = ribbon_vol_ax2.plot(data.index, data['BTC-USD_volatility_20'], label='20 Day Vol', linestyle='--', linewidth=2)

pnl_cond = (data['BTC-USD_trend_trade'].notnull())
_ = pnl_ax.plot(data[pnl_cond].index, data[pnl_cond]['BTC-USD_mavg_trend_PnL_cum'], label='Cumulative P&L', linestyle='--', linewidth=2)

# _ = trade_ax.plot(data.index, data['BTC-USD'], label='Price', linestyle='--', color='magenta', linewidth=2)
# buy_cond = (data['BTC-USD_trend_slope_signal'] == 1)
# sell_cond = (data['BTC-USD_trend_slope_signal'] == -1)
# _ = slope_ax.plot(data[buy_cond].index, data[buy_cond]['BTC-USD_trend_slope_trade'], label='Buy', linestyle='', color='red', linewidth=25, marker='^')
# _ = slope_ax.plot(data[sell_cond].index, data[sell_cond]['BTC-USD_trend_slope_trade'], label='Sell', linestyle='', color='green', linewidth=25, marker='v')

_ = mavg_ax.grid()
_ = mavg_ax.legend(ncol=4)
_ = mavg_ax.set_title('BTC')

# _ = slope_ax.grid()
# _ = slope_ax.legend(ncol=4, loc='upper left')
# _ = slope_ax2.legend(ncol=4, loc='upper right')
# _ = slope_ax.set_title('BTC Trend Signal')

_ = signal_ax.grid()
_ = signal_ax.legend(ncol=4)
_ = signal_ax.set_title('BTC Signals')

_ = ribbon_vol_ax.grid()
_ = ribbon_vol_ax.legend(ncol=4, loc='upper left')
_ = ribbon_vol_ax2.legend(ncol=4, loc='upper right')
_ = ribbon_vol_ax.set_title('BTC Ribbon Thickness')

_ = pnl_ax.grid()
_ = pnl_ax.legend(ncol=4)
_ = pnl_ax.set_title('BTC Cumulative P&L')

plt.tight_layout()

In [ ]:
data['BTC-USD'].corr(data['BTC-USD_ribbon_thickness'])

In [ ]:
data['BTC-USD'].corr(data['BTC-USD_ribbon_thickness_slope'])

In [ ]:
data.iloc[i-0]['BTC-USD_ribbon_thickness_diff']

In [ ]:
all(data.loc[data.index[i-j], 'BTC-USD_ribbon_thickness_diff'] > 0 for j in range(1,4))

In [ ]:
data.loc[data.index[i-0], 'BTC-USD_ribbon_thickness_diff']# > 0 for j in range(1,4)

In [ ]:
data.loc[data.index[i:]]

In [ ]:
data#.shape

In [ ]:
data.groupby(['BTC-USD_ribbon_thickness_position']).size()

In [ ]:
fig = plt.figure(figsize=(20,15))
# plt.style.use('bmh')
layout = (4,2)
mavg_ax = plt.subplot2grid(layout, (0,0), colspan=2)
ribbon_vol_ax = plt.subplot2grid(layout, (2,0), colspan=2)
signal_ax = plt.subplot2grid(layout, (1,0), colspan=2)
pnl_ax = plt.subplot2grid(layout, (3,0), colspan=2)
# trade_ax = plt.subplot2grid(layout, (1,1))

# _ = mavg_ax.plot(data.index, data['ETH-USD_short_mavg'], label='Short MAVG', linestyle='--', color='blue')
# _ = mavg_ax.plot(data.index, data['ETH-USD_long_mavg'], label='Long MAVG', linestyle='--', color='orange')
# _ = ribbon_ax.plot(data.index, data['ETH-USD_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='magenta', linewidth=2)
_ = mavg_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta', linewidth=1, alpha=0.6)
for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
    _ = mavg_ax.plot(data.index, data[f'ETH-USD_{int(window)}_mavg'], label=f'{int(window)} MAVG', linestyle='--')
    
buy_cond = (data['ETH-USD_trend_signal_diff'] == 1)
sell_cond = (data['ETH-USD_trend_signal_diff'] == -1)
_ = mavg_ax.plot(data[buy_cond].index, data[buy_cond]['ETH-USD_trend_trade'], label='Buy', linestyle='', color='cyan', linewidth=50, marker='^')
_ = mavg_ax.plot(data[sell_cond].index, data[sell_cond]['ETH-USD_trend_trade'], label='Sell', linestyle='', color='red', linewidth=50, marker='v')
    
# _ = slope_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta', linewidth=2, alpha=0.6)
# slope_ax2 = slope_ax.twinx()
# for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
#     _ = slope_ax2.plot(data.index, data[f'ETH-USD_{window}_mavg_slope'], label=f'{window} MAVG Slope', linestyle='--')

_ = signal_ax.plot(data.index, data['ETH-USD_trend_signal'], label='Trend Signal', linestyle='--', color='orange', linewidth=2)
# _ = signal_ax.plot(data.index, data['ETH-USD_trend_slope_signal'], label='Trend Slope Signal', linestyle='--', color='green', linewidth=2)#, marker='v')

_ = ribbon_vol_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta', linewidth=1, alpha=0.6)
ribbon_vol_ax2 = ribbon_vol_ax.twinx()
_ = ribbon_vol_ax2.plot(data.index, data['ETH-USD_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='red', linewidth=2)
# _ = ribbon_vol_ax2.plot(data.index, data['ETH-USD_volatility_10'], label='10 Day Vol', linestyle='--', linewidth=2)
# _ = ribbon_vol_ax2.plot(data.index, data['ETH-USD_volatility_20'], label='20 Day Vol', linestyle='--', linewidth=2)

pnl_cond = (data['ETH-USD_trend_trade'].notnull())
_ = pnl_ax.plot(data[pnl_cond].index, data[pnl_cond]['ETH-USD_mavg_trend_PnL_cum'], label='Cumulative P&L', linestyle='--', linewidth=2)

# _ = trade_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta', linewidth=2)
# buy_cond = (data['ETH-USD_trend_slope_signal'] == 1)
# sell_cond = (data['ETH-USD_trend_slope_signal'] == -1)
# _ = slope_ax.plot(data[buy_cond].index, data[buy_cond]['ETH-USD_trend_slope_trade'], label='Buy', linestyle='', color='red', linewidth=25, marker='^')
# _ = slope_ax.plot(data[sell_cond].index, data[sell_cond]['ETH-USD_trend_slope_trade'], label='Sell', linestyle='', color='green', linewidth=25, marker='v')

_ = mavg_ax.grid()
_ = mavg_ax.legend(ncol=4)
_ = mavg_ax.set_title('ETH')

# _ = slope_ax.grid()
# _ = slope_ax.legend(ncol=4, loc='upper left')
# _ = slope_ax2.legend(ncol=4, loc='upper right')
# _ = slope_ax.set_title('ETH Trend Signal')

_ = signal_ax.grid()
_ = signal_ax.legend(ncol=4)
_ = signal_ax.set_title('ETH Signals')

_ = ribbon_vol_ax.grid()
_ = ribbon_vol_ax.legend(ncol=4)
_ = ribbon_vol_ax.legend(ncol=4, loc='upper left')
_ = ribbon_vol_ax2.legend(ncol=4, loc='upper right')
_ = ribbon_vol_ax.set_title('ETH Ribbon Thickness')

_ = pnl_ax.grid()
_ = pnl_ax.legend(ncol=4)
_ = pnl_ax.set_title('ETH Cumulative P&L')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(20,15))
# plt.style.use('bmh')
layout = (3,2)
mavg_ax = plt.subplot2grid(layout, (0,0), colspan=2)
ribbon_vol_ax = plt.subplot2grid(layout, (2,0), colspan=2)
signal_ax = plt.subplot2grid(layout, (1,0), colspan=2)
# trade_ax = plt.subplot2grid(layout, (1,1))

# _ = mavg_ax.plot(data_SPY.index, data_SPY['SPY_short_mavg'], label='Short MAVG', linestyle='--', color='blue')
# _ = mavg_ax.plot(data_SPY.index, data_SPY['SPY_long_mavg'], label='Long MAVG', linestyle='--', color='orange')
# _ = ribbon_ax.plot(data_SPY.index, data_SPY['SPY_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='magenta', linewidth=2)
_ = mavg_ax.plot(data_SPY.index, data_SPY['SPY'], label='Price', linestyle='--', color='magenta', linewidth=1, alpha=0.6)
for window in np.linspace(mavg_start_SPY, mavg_end_SPY, mavg_stepsize_SPY):
    _ = mavg_ax.plot(data_SPY.index, data_SPY[f'SPY_{window}_mavg'], label=f'{window} MAVG', linestyle='--')
    
buy_cond = (data_SPY['SPY_trend_signal_diff'] == 1)
sell_cond = (data_SPY['SPY_trend_signal_diff'] == -1)
_ = mavg_ax.plot(data_SPY[buy_cond].index, data_SPY[buy_cond]['SPY_trend_trade'], label='Buy', linestyle='', color='cyan', linewidth=50, marker='^')
_ = mavg_ax.plot(data_SPY[sell_cond].index, data_SPY[sell_cond]['SPY_trend_trade'], label='Sell', linestyle='', color='red', linewidth=50, marker='v')
    
# _ = slope_ax.plot(data_SPY.index, data_SPY['SPY'], label='Price', linestyle='--', color='magenta', linewidth=2, alpha=0.6)
# slope_ax2 = slope_ax.twinx()
# for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
#     _ = slope_ax2.plot(data_SPY.index, data_SPY[f'SPY_{window}_mavg_slope'], label=f'{window} MAVG Slope', linestyle='--')

_ = signal_ax.plot(data_SPY.index, data_SPY['SPY_trend_signal'], label='Trend Signal', linestyle='--', color='orange', linewidth=2)
# _ = signal_ax.plot(data_SPY.index, data_SPY['SPY_trend_slope_signal'], label='Trend Slope Signal', linestyle='--', color='green', linewidth=2)#, marker='v')

_ = ribbon_vol_ax.plot(data_SPY.index, data_SPY['SPY_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='red', linewidth=2)
ribbon_vol_ax2 = ribbon_vol_ax.twinx()
_ = ribbon_vol_ax2.plot(data_SPY.index, data_SPY['SPY_volatility_10'], label='10 Day Vol', linestyle='--', linewidth=2)
_ = ribbon_vol_ax2.plot(data_SPY.index, data_SPY['SPY_volatility_20'], label='20 Day Vol', linestyle='--', linewidth=2)

# _ = trade_ax.plot(data_SPY.index, data_SPY['SPY'], label='Price', linestyle='--', color='magenta', linewidth=2)
# buy_cond = (data_SPY['SPY_trend_slope_signal'] == 1)
# sell_cond = (data_SPY['SPY_trend_slope_signal'] == -1)
# _ = slope_ax.plot(data_SPY[buy_cond].index, data_SPY[buy_cond]['SPY_trend_slope_trade'], label='Buy', linestyle='', color='red', linewidth=25, marker='^')
# _ = slope_ax.plot(data_SPY[sell_cond].index, data_SPY[sell_cond]['SPY_trend_slope_trade'], label='Sell', linestyle='', color='green', linewidth=25, marker='v')

_ = mavg_ax.grid()
_ = mavg_ax.legend(ncol=4)
_ = mavg_ax.set_title('SPY')

_ = slope_ax.grid()
_ = slope_ax.legend(ncol=4, loc='upper left')
_ = slope_ax2.legend(ncol=4, loc='upper right')
_ = slope_ax.set_title('SPY Trend Signal')

_ = signal_ax.grid()
_ = signal_ax.legend(ncol=4)
_ = signal_ax.set_title('SPY Signals')

_ = ribbon_vol_ax.grid()
_ = ribbon_vol_ax.legend(ncol=4)
_ = ribbon_vol_ax.set_title('SPY Ribbon Thickness')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(12,6))
# plt.style.use('bmh')
layout = (1,1)
mavg_ax = plt.subplot2grid(layout, (0,0))

_ = mavg_ax.plot(data.index, data['BTC-USD'], label='Price', linestyle='--', color='magenta')
mavg_ax2 = mavg_ax.twinx()
_ = mavg_ax2.plot(data.index, data['BTC_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='red')
# for window in np.linspace(10, 100, 10):
#     _ = mavg_ax.plot(data.index, data[f'BTC_{window}_mavg'], label=f'{window} MAVG', linestyle='--')

_ = mavg_ax.grid()
_ = mavg_ax.legend(loc='upper left')
_ = mavg_ax2.legend(loc='upper right')
_ = mavg_ax.set_title('BTC')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(12,6))
# plt.style.use('bmh')
layout = (1,1)
mavg_ax = plt.subplot2grid(layout, (0,0))

# _ = mavg_ax.plot(data.index, data['BTC_short_mavg'], label='Short MAVG', linestyle='--', color='blue')
# _ = mavg_ax.plot(data.index, data['BTC_long_mavg'], label='Long MAVG', linestyle='--', color='orange')
_ = mavg_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta', linewidth=3)
for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
    _ = mavg_ax.plot(data.index, data[f'ETH_{window}_mavg'], label=f'{window} MAVG', linestyle='--')

_ = mavg_ax.grid()
_ = mavg_ax.legend(ncol=3)
_ = mavg_ax.set_title('ETH')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(12,6))
# plt.style.use('bmh')
layout = (1,1)
mavg_ax = plt.subplot2grid(layout, (0,0))

_ = mavg_ax.plot(data.index, data['ETH-USD'], label='Price', linestyle='--', color='magenta')
mavg_ax2 = mavg_ax.twinx()
_ = mavg_ax2.plot(data.index, data['ETH_ribbon_thickness'], label='Ribbon Thickness', linestyle='--', color='red')
# for window in np.linspace(10, 100, 10):
#     _ = mavg_ax.plot(data.index, data[f'BTC_{window}_mavg'], label=f'{window} MAVG', linestyle='--')

_ = mavg_ax.grid()
_ = mavg_ax.legend()
_ = mavg_ax.set_title('ETH')

plt.tight_layout()

In [ ]:
col_list = [col for col in data.columns if (col[:3] == 'BTC') & (col[-4:] == 'mavg')]

In [ ]:
data[col_list].head()

In [ ]:
data.head()

In [ ]:
window_list = [window for window in np.linspace(10, 100, 10)]
for i in np.arange(len(window_list)-1):
    print(i, window_list[i], window_list[i+1])
    data[f'signal_{window_list[i]}_{window_list[i+1]}'] = np.where(data[f'BTC_{window_list[i]}_mavg'] >
                                                                  data[f'BTC_{window_list[i+1]}_mavg'], 1, 0)

In [ ]:
data.head(100)

In [ ]:
data['signal'] = np.where(data['BTC_short_mavg'] > data['BTC_long_mavg'], 1, -1)

In [ ]:
short_mavg = asset.rolling(window=30, center=False).mean()
long_mavg = asset.rolling(window=200, center=False).mean()

asset[200:].plot(figsize=(15,7))
short_mavg[200:].plot()
long_mavg[200:].plot()
plt.ylabel('Price')
plt.show()